In [2]:
import os
import azureml.core
from azureml.core import Workspace, Model
from azureml.core.conda_dependencies import CondaDependencies

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.19.0


# 파라미터

In [3]:
MODEL_NAME = "04.00.00"
MODEL_DESC = "train data 2020-01-08 - 20020-01-28 / JYTek / Ferrite Core Applied / Use Only Test 2, 3 / Notch Filter Applied /= 04.00.00"
IMG_DESC = "MCTFT 개선모델 최종버전에 테스트2,3 만 적용하도록 수정 / Feature 저장 기능 추가"
WORKSPACE = "develope" # develope, production

## 01 Azure ML Service 작업 영역에 연결

In [4]:
if WORKSPACE == "develope":
    ws_dir = "config/config_dev.json"
elif WORKSPACE == "production":
    ws_dir = "config/config.json"
    
ws = Workspace.from_config("config/config_dev.json")
print('기계학습이름:'+ws.name, '리소스그룹이름:'+ws.resource_group, '리전이름:'+ws.location, '구독ID:'+ws.subscription_id, sep = '\n')

기계학습이름:lsmc-dev-mlws
리소스그룹이름:lsmc-prd-rg
리전이름:koreacentral
구독ID:1f154107-bef0-481f-a741-68e7ba34affe


## 02 모델 등록

### 모델 가져오기

- 방법 1. models 경로에 직접 파일을 저장
- 방법 2.  개발과정에서 등록한 모델 가져오기

In [5]:
ws_model = Workspace.from_config("../.azureml/config.json")
print(ws_model.models)
model = Model(ws_model, '04.00.00', version=6)
model.download(target_dir='models/tmp/', exist_ok=True)
donwloaded_file = os.listdir('models/tmp/')[0]
os.rename(f"models/tmp/{donwloaded_file}", f"models/{MODEL_NAME}.pkl")
os.rmdir("models/tmp/")
print(f"\nDownloaded File: {donwloaded_file}\n")
print(f"is Modified to: {MODEL_NAME}.pkl\n")
file_name = f"{MODEL_NAME}.pkl"

{'04.00.00': Model(workspace=Workspace.create(name='lsmc-op-ml', subscription_id='1f154107-bef0-481f-a741-68e7ba34affe', resource_group='is-mc-analysis'), name=04.00.00, id=04.00.00:7, version=7, tags={'area': 'IoT Edge', 'type': 'azureml-automl'}, properties={})}

Downloaded File: model_v2.pkl

is Modified to: 04.00.00.pkl



### 모델 등록

In [7]:
from azureml.core.model import Model

#library_version = "DL"+sklearn.__version__.replace(".","x")
model = Model.register(model_path = f"models/{MODEL_NAME}.pkl",
                       model_name = MODEL_NAME,
                       tags = {'area': "IoT Edge", 'type': "azureml-automl"},
                       description = MODEL_DESC,
                       workspace = ws_model)

Registering model 04.00.00


In [8]:
model = Model.get_model_path(MODEL_NAME, _workspace=ws_model)
model

'azureml-models/04.00.00/8/04.00.00.pkl'

# 03 모델 실행 환경 세팅 파일(config/myenv.yml) 생성

In [35]:
pip_packages = ["numpy==1.18.5",
                "pandas==0.25.3",
                "sqlalchemy==1.3.21",
                "h5py==2.10.0",
                "tqdm==4.54.0",
                "obspy==1.2.2",
                "azure-storage-blob==12.6.0",
                "mysql-connector-python==8.0.18",
                "azureml==0.2.7",
                "azureml-core==1.19.0",
                "azureml-sdk==1.19.0",
                "scipy==1.6.0",
                "xgboost==0.90"]

lib_config_load = ['pyyaml']
lib_clfs = ["python=3.7.9",
            "gcc_linux-64"]

conda_packages = lib_clfs + lib_config_load

print('pip packages:', pip_packages)
print('conda packasges:', conda_packages)


myenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)
myenv.add_channel('conda-forge')
myenv.add_channel('defaults')

with open("config/myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

pip packages: ['numpy==1.18.5', 'pandas==0.25.3', 'sqlalchemy==1.3.21', 'h5py==2.10.0', 'tqdm==4.54.0', 'obspy==1.2.2', 'azure-storage-blob==12.6.0', 'mysql-connector-python==8.0.18', 'azureml==0.2.7', 'azureml-core==1.19.0', 'azureml-sdk==1.19.0', 'scipy==1.6.0', 'xgboost==0.90']
conda packasges: ['python=3.7.9', 'gcc_linux-64', 'pyyaml']


# 04 전처리 및 ML 판정 수행 로직 실행 스크립트(score_iot.py) 생성

In [6]:
import os
os.listdir("test")

FileNotFoundError: [Errno 2] No such file or directory: 'test'

In [ ]:
%%writefile score.py
def init():
    global loaded_model, model_object, model_version, column_names, line, cut_off, rule_model_params, model_path
    
    ## 수정필요  --------------------------------------------------------
    model_object = model_v03_01_00
    model_version = '03.01.00' #model_name 입력
    column_names = ["FTUR_CPSR_MAX_N1","FTUR_CPSR_MAX_N2","FTUR_CPSR_MAX_N3","FTUR_CPSR_MAX_N4",
                    "FTUR_CPSR_CRSTFT_N1","FTUR_CPSR_CRSTFT_N2","FTUR_CPSR_CRSTFT_N3","FTUR_CPSR_CRSTFT_N4"]
    ## ----------------------------------------------------------------
    file_name = '{}.pickle'.format(model_version)
    edge_config = '/home/data/edge_config.yml'
        
    #load model
    model_path = Model.get_model_path(file_name) 
    with open(model_path, "rb") as f:
        loaded_model = pickle.load(f)
        
    #load line info
    with open(edge_config , 'r') as stream:
        try:
            edge_config = yaml.load(stream, Loader=yaml.BaseLoader)
            line = edge_config['config']['line']['name']
            cut_off = float(edge_config['config']['param']['cutoff'])
            rule_model_params = edge_config['config']['param']['rule_model']
            
        except yaml.YAMLError as exc:
            print("line config error: ",exc)
    
    error_dir = '/home/data/error_file/'
    try:
        os.makedirs(error_dir, mode = 777)
    except:
        pass

def run(input_json):
    print('\n','mlmodule start')
    print('\n',datetime.now()+ timedelta(hours=9),'\n')
    #for test#
    input_json = json.loads(input_json)
    print('\n','json loaded','\n')
    print(input_json, '\n')
    mltime = datetime.now() + timedelta(hours=9)
    chtime = input_json['chtime']
    print('chtime : ', chtime)
    ct = datetime.strptime(chtime.replace('T',' ').split('+')[0][:-1], '%Y-%m-%d %H:%M:%S.%f')
    diff = mltime-ct
    input_json['chtime'] = str(ct)
    input_json['mltime'] = str(mltime)
    input_json['etime_ch'] = diff.seconds + diff.microseconds/1E6
       
    
    #file load
    init_time = time.time()
    #input_json = json.loads(input_json)
    input_path = input_json['path']
    print('\n', input_path)
    
    for attempt in range(3):
        try:
            print('Attempt: ' + str(attempt+1) + '  Time: ' + str(datetime.now()))
            with h5py.File(input_path, 'r') as f:
                tmp = f['Raw'][:]
            input_raw = pd.DataFrame(tmp, columns=['x1','x2','x3'])
            print('\n', 'hdf loaded')

        except Exception as ex: 
            if attempt == 2:
                ex_message = str(ex)
            
            sleep(0.02)
            continue
        
        break
        
    else:    
        input_json['b'] = 2
        input_json['prob'] = 2
        input_json['error'] = ex_message
        input_json['etime'] = time.time() - init_time

        result_json =  [json.dumps(input_json)]
        print('*'*5,' ','LOAD ERROR',' ','*'*5)
        print(result_json)
        try:
            shutil.copy(input_path, error_dir)
        except:
            pass
    
    #for test#
    #diff = (datetime.now()+ timedelta(hours=9))-mltime
    #input_json['etime_load'] = diff.seconds + diff.microseconds/1E6
    input_json['etime_load'] =time.time() - init_time
    
    ### json insert
    input_json['cutoff'] = cut_off
    input_json['TRHD_NM_SET'] = "/".join([key for key in rule_model_params.keys()])
    input_json['TRHD_VAL_SET'] = "/".join([str(value) for value in rule_model_params.values()])
    input_json['FTUR_NM_SET'] = "/".join(["/".join([col+'_TEST_N1' for col in column_names]),
                                          "/".join([col+'_TEST_N2' for col in column_names]),
                                          "/".join([col+'_TEST_N3' for col in column_names])])
    input_json['lid'] = line
    input_json['v'] = model_version
    
    try:
        filename = input_path.split('/')[-1]
        filename = filename.strip('Data\\').split('_')
        input_json['bc'] = '_'.join(filename[0:2])
        input_json['dtfull'] = filename[2][:-3]

    except Exception as ex: # 에러 종류
        input_json['bc'] = 'ERROR'
        input_json['dtfull'] = str(datetime.now())
        input_json['r'] = 2
        input_json['prob'] = 2
        input_json['error'] = str(ex)
        input_json['etime'] = time.time() - init_time

        result_json =  [json.dumps(input_json)]
        print('*'*5,' ','FILENAME ERROR',' ','*'*5)
        print(result_json)
        try:
            cmd = 'cp ' + input_json['path'] + ' ' + error_dir
            os.system(cmd)
        except:
            pass
        
    ##-------------Inputs-----------------
    md = model_object(cut_off=cut_off,
                     rule_params={'FeatureEnergy':{'cut_off':float(rule_model_params['FTUR_ENRG_TRHD'])},
                                  'FeatureWaveWindowedSTDMin':{'cut_off':float(rule_model_params['FTUR_WVFM_STDDEV_TRHD'])},
                                  'FeatureTriggers':{'cut_off':float(rule_model_params['FTUR_TRGER_TRHD'])}},
                     model = loaded_model)

    try:
        data = pd.DataFrame(input_raw.transpose(), columns=['x1','x2','x3'])
        _, pred_y, pred_prob, error, _, data_model = md.run_classifier(input_raw)
        input_json['r'] = int(pred_y)
        input_json['prob'] = float(pred_prob)
        input_json['error'] = str(error)
        input_json['etime'] = time.time() - init_time
        input_json['FTUR_VAL_SET'] = "/".join([str(value) for value in data_model.flatten()])
        
        result_json =  [json.dumps(input_json)]
        print(result_json)
    
    except Exception as ex: # 에러 종류
        input_json['r'] = 2
        input_json['prob'] = 2
        input_json['error'] = str(ex)
        input_json['etime'] = time.time() - init_time
        input_json['FTUR_VAL_SET'] = ""

        result_json =  [json.dumps(input_json)]
        print('*'*5,' ','ERROR',' ','*'*5)
        print(result_json)
        try:
            cmd = 'cp ' + input_json['path'] + ' ' + error_dir
            os.system(cmd)
        except:
            pass
        
    
    return result_json

In [36]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_mnist_model.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Writing score.py


# 05 score_iot.py 테스트

# 06 Azure MLWS 이미지 생성 및 업로드

In [ ]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                                  execution_script="score_iot.py",
                                                  conda_file="data/myenv.yml",
                                                  dependencies="../src",
                                                  tags = {'area': "IoT Edge", 'type': "azure-automl"},
                                                  description = IMG_DESC)

image = Image.create(name = "mc-mlmodule",
# image = Image.create(name = "mc-namtest",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

# 07 이미지 빌드

In [ ]:
image.wait_for_creation(show_output = True)

# 08 빌드 로그 확인 방법

In [ ]:
for i in Image.list(workspace = ws,tags = ["area"]):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/tutorials/img-classification-part2-deploy.png)